# VacationPy
----



In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_path = "../output_data/cities.csv"
cities_pd = pd.read_csv(cities_path)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
max_humidity = cities_pd['Humidity'].max()
gmaps.configure(api_key=g_key)
locations = cities_pd[["Lat", "Lng"]].astype(float)
humidity_rate = cities_pd["Humidity"].astype(float)


In [4]:
figure_layout = {
    'width': '700px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate,
                                dissipating=False, max_intensity=max_humidity,
                                point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_pd = cities_pd.dropna()
good_cities_df = cities_pd[(cities_pd['Max_Temp'].between(66, 76)) &
                           (cities_pd['Wind_Speed']<10) & 
                           (cities_pd['Cloudiness']==0)].dropna()


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 5000
target_type = "lodging"

for ind in range (len(good_cities_df)):
    lat = good_cities_df.iloc[ind]['Lat']
    lng = good_cities_df.iloc[ind]['Lng']
    params = {
        "location": f"{lat},{lng}",
        "radius": target_radius,
        "type": target_type,
        "key": g_key
        }
    response = requests.get(base_url, params=params)
    places_data = response.json()
    try:
        hotel_name = places_data['results'][0]['name']
        hotel_df.append(hotel_name)
    except:
        hotel_df.append("****")

In [7]:
good_cities_df['Hotel Name'] = hotel_df


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in good_cities_df.iterrows()]
locations = good_cities_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
max_humidity = good_cities_df['Humidity'].max()
humidity_rate = good_cities_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate,
                                dissipating=False, max_intensity=max_humidity,
                                point_radius = 1)

fig.add_layer(heat_layer)

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(markers)

fig



Figure(layout=FigureLayout(height='420px'))